In [4]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
#%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
import tensorflow as tf
from ResNet50 import *
from LoadDataset import *
from tensorboard import notebook

In [3]:
a = tf.Variable(2)
tf.print(a)
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:
   print("Please install GPU version of TF")

2
Default GPU Device:/device:GPU:0


In [19]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import matplotlib.pyplot as plt
import datetime

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Reshape
from tensorflow.keras import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow.keras.backend as K
from Squeeze_and_Excite import Squeeze_and_Excite 
from ResNet50 import *
from LoadDataset import *

#%load_ext tensorboard
#!rm -rf ./logs/

epoch = 100
batch_size = 128
learning_rate = 0.01
dataset = 100


def oneHotEncoder(data):
    label_encoder = LabelEncoder()
    values = np.array(data)
    onehot_encoder = OneHotEncoder(sparse = False)
    integer_encoded = label_encoder.fit_transform(values)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
test_top1 = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='test_1_accuracy') 
test_top5 = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='test_5_accuracy') 

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training = True)
        loss =loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
        
@tf.function
def test_step(images, labels):
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training = False)
    t_loss =loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    test_top1(labels, predictions)
    test_top5(labels, predictions)


if __name__ == "__main__":

    save_model_dir = "./checkpoints_pre"

    print("loading data 2...")
    train_X, train_lab, test_X, test_lab = get_data(dataset)
    print("normalizing data...")
    train_X, test_X = normalize(train_X, test_X)
    train_data = tf.data.Dataset.from_tensor_slices((train_X, train_lab)).batch(batch_size)
    test_data = tf.data.Dataset.from_tensor_slices((test_X, test_lab)).batch(batch_size)

    #model = ResNet50(include_top=True, weights=None, squeeze=False, squeeze_type='Normal')#, input_tensor = tf.data.Dataset.from_tensor_slices((train_X)))
    #input_tensor = tf.placeholder(tf.float32, shape = [None, train_X.shape[0], train_X.shape[1], train_X.shape[2]])
    model = ResNet50(include_top=True, squeeze=False, squeeze_type='normal', classes=dataset) #pre, identity, normal

    """
    features, label = iter(train_dataset).next()
    print("example features:", features[0])
    print("example label:", label[0])
    """

    checkpoint_dir = os.path.join(save_model_dir, "ckpt")
    checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
    manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=3)
    checkpoint.restore(manager.latest_checkpoint)
    #if manager.latest_checkpoint:
    #    print("Restaurado de {}".format(manager.latest_checkpoint))
    #else:
    #    print("Inicializando desde cero")

    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
    test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)

loading data 2...


TypeError: can only concatenate str (not "list") to str

In [17]:
%tensorboard --logdir train_log_dir
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir train_log_dir (started 0:00:00 ago; pid 7867)


In [ ]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

In [ ]:
   train_losses = []
    train_accs = []
    test_losses = []
    test_accs = []

    for i in range(epoch):
        print(i)
        for images, labels in train_data:
            train_step(images, labels)
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=i+1)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=i+1)

            
        for images, labels in test_data:
            test_step(images, labels)
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=i+1)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=i+1)

        template = 'Epoch {}, Train Loss: {}, Train Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print(template.format(i+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

        template = 'Top1 Error: {}, Top5 Error: {}'
        print(template.format((1 - test_top1.result())*100,\
                            (1 - test_top5.result())*100))

        save_path = manager.save()

        train_losses.append(train_loss.result())
        train_accs.append(train_accuracy.result())
        test_losses.append(test_loss.result())
        test_accs.append(test_accuracy.result())

        # Reinicia las metricas para el siguiente epoch.
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()


    t = np.linspace(1, epoch, num=epoch)
    plot1 = plt.figure(1)
    plt.plot(t, train_losses, 'b')
    plt.plot(t, test_losses, 'r')
    plt.xlabel('epoch')
    plt.ylabel('Loss')
    plt.title('Training loss evolution')
    plt.savefig('./Result_Pics/loss_train')
    plot2 = plt.figure(2)
    plt.xlabel('epoch')
    plt.ylabel('Accuracy')
    plt.plot(t, train_accs, 'b')
    plt.plot(t, test_accs, 'r')
    plt.title('Training accuracy evolution')
    plt.savefig('.//Result_Pics//acc_train')
    plt.show()
    
    a = 1